# Python Asynchronous I/O Tutorial

In [1]:
import aiohttp
import asyncio
import time
import requests
import multiprocessing
from multiprocessing import Pool
from multiprocessing.dummy import Pool as ThreadPool

In [2]:
urls = [
    "https://www.google.com",
    "https://www.microsoft.com",
    "https://www.twitter.com",
    "https://www.netflix.com",
    "https://www.facebook.com",
    "https://www.apple.com",
    "https://www.amazon.com",
    "https://www.reddit.com",
    "https://www.sony.com",
    "https://www.nintendo.com",
    "https://www.oracle.com",
    "https://www.intel.com",
    "https://www.samsung.com",
    "https://www.ibm.com",
]

In [3]:
# fix needed for Python 3.8+ on mac
# alternative is to use the 'multiprocess' library 
multiprocessing.set_start_method('fork')

## Sequential

In [4]:
start_time = time.time()

results = []
for each_url in urls:
    try:
        each_response = requests.get(each_url)
        each_content = each_response.content
        results.append(each_content)
    except Exception as e:
        print(f"{each_url} {e}")
        results.append(None)
    
time.time() - start_time

9.155354976654053

## Multi-Processing

In [5]:
start_time = time.time()

def example_multiprocessing_function(each_url):
    try:
        each_response = requests.get(each_url)
        each_content = each_response.content
        return each_content
    except Exception as e:
        print(f"{each_url} {e}")
        return None

with Pool(processes=multiprocessing.cpu_count()) as pool:
    results = pool.map(example_multiprocessing_function, urls)
    
time.time() - start_time

2.842160940170288

## Multi-Threading

In [6]:
start_time = time.time()

with ThreadPool(processes=multiprocessing.cpu_count()) as pool:
    results = pool.map(example_multiprocessing_function, urls)
    
time.time() - start_time

4.485712051391602

## Asyncio

### General Example

In [7]:
start_time = time.time()

# 1. add "async" to def to create a coroutine
# 2. use "await" for concurrent/coroutine calls
# note: await calls and packages must support asyncio 
async def example_asyncio_function(x):
    await asyncio.sleep(x)
    return x

# 3. create concurrent main coroutine
# 4. use asyncio.create_task() to create tasks
# 5. use asyncio.gather() to schedule concurrent tasks
# note: can also use asyncio.wait() and asyncio.wait_for()
async def concurrent_main_function():
    inputs = [1, 2, 3, 4, '5', 6]
    tasks = []
    for each_input in inputs:
        each_coroutine = example_asyncio_function(each_input)
        each_task = asyncio.create_task(each_coroutine)
        tasks.append(each_task)
    results = await asyncio.gather(*tasks, return_exceptions=True)
    return results

# 6. await main coroutine 
# note: if outside of a notebook, use asyncio.run()
results = await concurrent_main_function()

time.time() - start_time

6.003752946853638

### HTTP Example

In [8]:
start_time = time.time()

async def example_asyncio_get_request(session, each_url):
    async with session.get(each_url) as resp:
        result = await resp.read()
        return result  

async def main():
    async with aiohttp.ClientSession() as session:
        tasks = []
        for each_url in urls:
            each_coroutine = example_asyncio_get_request(session, each_url)
            each_task = asyncio.create_task(each_coroutine)
            tasks.append(each_task)
        results = await asyncio.gather(*tasks, return_exceptions=True)
        return results

results = await main()

time.time() - start_time

1.4870429039001465